In [3]:
import json
from bs4 import BeautifulSoup
import requests
import time
import csv

from selenium import webdriver

In [4]:
def getSeattleListingURLSinglePg(minPrice, maxPrice, itemOffset, listingList):
    base_url = 'https://www.airbnb.com/s'
    search_city = '/Seattle--WA--United-States/homes?'
    refinements = 'refinement_paths%5B%5D=%2Fhomes&query=Seattle%2C%20WA&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&s_tag=Dg4jrsse&section_offset=7&items_offset={}&price_min={}&price_max={}&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Private%20room'.format(itemOffset, minPrice, maxPrice)
    url =  base_url + search_city + refinements
    
    # put the path to chromedriver
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(url) 
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    listings = soup.find_all('meta', itemprop = 'url')
    
    for listing in listings:
        url = listing['content'].split('?')[0]
        listingList.append(url)
    
    driver.quit()
    return listingList

In [5]:
def getMaxNumPg(minPrice, maxPrice):
    base_url = 'https://www.airbnb.com/s'
    search_city = '/Seattle--WA--United-States/homes?'
    refinements = 'refinement_paths%5B%5D=%2Fhomes&query=Seattle%2C%20WA&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&s_tag=Dg4jrsse&section_offset=7&items_offset=0&price_min={}&price_max={}&room_types%5B%5D=Entire%20home%2Fapt&room_types%5B%5D=Private%20room'.format(minPrice, maxPrice)
    url =  base_url + search_city + refinements
    
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(url) 
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    pages = (soup.find('ul', class_ = '_11hau3k')).find_all('li')
#     pages = soup.find_all('li')
    lastPage = (((pages[4])['data-id']).split('-'))[1]

    return int(lastPage)

In [6]:
def getSeattleListingURLMultiPg(minPrice, maxPrice, numPg, listingList):
    ##get max number of page from the search result
#     maxPg = getMaxNumPg(minPrice, maxPrice)
    
#     totalPg = min(maxPg, numPg) # get total number of pages to get
    totalPg = numPg
    currentPg = 1 # starting at page 1
    while currentPg <= totalPg:
        currentItemOffset = (currentPg-1) * 18    #18 items per page
        listingList = getSeattleListingURLSinglePg(minPrice, maxPrice, currentItemOffset, listingList)
        
        currentPg += 1
        
    return listingList


In [12]:
def getSeattleListingURLMultiPrice(overallMinPrice, overallMaxPrice, listingList):
    ## listings for every 5 dollar price cut
    priceRange = range(overallMinPrice, overallMaxPrice+1, 5)
    
    for i in range(len(priceRange) - 1):
        minPrice = priceRange[i] + 1 if i > 0 else priceRange[i]
        maxPrice = priceRange[i + 1]
    
        ## for each price range get 12 pages
        print(minPrice)
        print(maxPrice)
        listingList = getSeattleListingURLMultiPg(minPrice, maxPrice, 8, listingList)
        time.sleep(120) ##sleep 2 minutes 
         
    return listingList

In [10]:
def writeListingURLs(results, pathToWrite):
    with open(pathToWrite, 'a') as f:
        for item in results:
            f.write("%s\n" % item)      
    f.close()

In [15]:
overallMinPrice = 50
overallMaxPrice = 180
seattleAirbnbListings = []
seattleAirbnbListings = getSeattleListingURLMultiPrice(overallMinPrice, overallMaxPrice, seattleAirbnbListings)

pathToWrite = './data/listingURL.csv'
writeListingURLs(seattleAirbnbListings, pathToWrite)